# Steady-State Flow Towards a Well in an Unconfined Aquifer 

## Thiem Equation 

You can calculate the flow towards a well in an unconfinded Aquifer with this Equations:

$$R = 3000 \cdot s \cdot \sqrt{K}$$ caution: not true to unit, s in m and K in m/s  

$$Q = \pi K\frac{ (H^2 - h^2)}{ln(\frac{R}{r})}$$

with: <br>
    $R$ = radius of the influence [L], <br>
    $s$ = drawdown in the well [L], <br>
    $K$ = hydraulic conductivity [L/T], <br>
    $Q$ = pumping rate [L^3/T], <br>
    $H$ = water level at rest [L], <br>
    $h$ = water level at distance r, [L] <br>
    $r$ = distance from the well, [L] 
    
if you change the formula you get the following equation for the water level depending on the distance of the well:

$$h = \sqrt{H^2 - \frac{Q}{\pi \cdot K} \cdot ln\frac{R}{r}}$$

In [ ]:
# Importing necessary libraries
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import *

# Function to calculate the hydraulic head h in an unsaturated aquifer as a function of distance r

def compute_R(Q,K,H,r_w):
    # Initial guess for h (starting at H) and R
    h = H/2       
    max_it = 1000  # Limit on iterations to prevent infinite loops
    tol = 1e-6     # Convergence tolerance
    for iteration in range(max_it):
        R = 3000 * (H-h) * np.sqrt(K)
        h_new = np.sqrt (H**2 - (Q  / (np.pi * K) * np.log(R / r_w)))        
        # Check for convergence
        if np.abs(h_new - h) < tol:
            break     
        # Set head
        h = h_new    
    R = 3000 * (H-h) * np.sqrt(K)
    return R, h

def compute_h(Q,K,H,R,r):
    h = np.sqrt (H**2 - (Q  / (np.pi * K) * np.log(R / r)))
    # If outside of the range, keep H
    return h
    
def calculate_h_unsaturated(Q, K, H, r_w, x_max):
    #run the initial iteration to get R
    R, h_w = compute_R(Q,K,H,r_w)
   # print('Range of drawdown: ', R, ' with well head: ', h_w)

#COMPUTE h(r)
    r = np.arange(r_w, R, 0.01) 
    rm = r*-1
    h = compute_h(Q, K, H, R, r)

    #PLOT    
    fig = plt.figure(figsize=(10,5))
    ax = fig.add_subplot(1, 1, 1)
    
    ax.plot(r,h, '--' 'b')
    ax.plot(rm,h, '--' 'b')
    ax.hlines(y= H, xmin=R,  xmax= x_max, linestyle='--', colors='b')
    ax.hlines(y= H, xmin=-R, xmax=-x_max, linestyle='--', colors='b')
    
    ax.fill_between(r, h, 0, facecolor= 'lightblue')
    ax.fill_between(-r, h, 0, facecolor = 'lightblue')
    ax.axvspan(xmin= R, xmax= x_max, ymin=0, ymax=H/(H+5), color='lightblue')        
    ax.axvspan(xmin= -x_max, xmax= -R, ymin=0, ymax=H/(H+5),  color='lightblue')
    ax.axvspan(xmin= -r_w, xmax= r_w, ymin=0, ymax=h_w/(H+5),  color='lightblue')
    
    
    
    
    ax.set(xlabel='x [m]',ylabel='head [m]', xlim=[(-x_max*1.1),(x_max*1.1)], ylim=[0,(H+5)])
    
    # MAKE 'WATER'-TRIANGLE
    ax.arrow(x_max*0.95,(H+(H*0.04)), 0, -0.01, fc="k", ec="k",head_width=(x_max*0.04), head_length=(H*0.04))
    ax.hlines(y= H-(H*0.02), xmin=x_max*0.92, xmax=x_max*0.98, colors='blue')   
    ax.hlines(y= H-(H*0.04), xmin=x_max*0.94, xmax=x_max*0.96, colors='blue')  
    ax.text((x_max/2),1,'unconfined aquifer')
    
    plt.show()

interact(calculate_h_unsaturated,
         x_max = widgets.FloatSlider   (value = 2000,        min = 50,    max = 10000, step = 50,   description = r'\(\ x_{plot} \):', disabled = False),
         H     = widgets.FloatSlider   (value =15,           min = 1,     max = 100,   step = 0.5,  description='H:', disabled=False),
         r_w   = widgets.FloatSlider   (value =0.3,          min = 0.02,  max = 0.5,   step = 0.01, description=r'\(\ r_w\):', disabled=False),
         K     = widgets.FloatLogSlider(value =0.001,base=10,min = -8,    max = -1,    step = 0.01, readout_format='.8f', description='K:' ),
         Q     = widgets.FloatSlider   (value =0.01,         min = 0.001, max = 0.5,   step = 0.01, description='Q:' , disabled=False))